In [ ]:
%pip install azure-ai-documentintelligence==1.0.0b4
%pip install azure-ai-formrecognizer

In [ ]:
"""
This code sample shows Prebuilt Layout operations with the Azure Form Recognizer client library. 
The async versions of the samples require Python 3.6 or later.

To learn more, please visit the documentation - Quickstart: Document Intelligence (formerly Form Recognizer) SDKs
https://learn.microsoft.com/azure/ai-services/document-intelligence/quickstarts/get-started-sdks-rest-api?pivots=programming-language-python
"""
import json
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest


"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""
endpoint = ""
key = ""

formUrl = ""
document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)
    
poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-layout", formUrl)
result = poller.result()


response_content=result.content



In [ ]:
response_content

In [ ]:
import re
import json


def parse_immigration_law(text):
    """
    Function to parse immigration law text into structured JSON format.
    
    Args:
        text (str): The content of the immigration law document.

    Returns:
        list: A list of dictionaries, each representing a parsed article with its metadata.
    """
    # Define regex patterns for chapters, sections, and articles
    chapter_pattern = re.compile(r'^제(\d+)장\s+(.+)', re.MULTILINE)
    section_pattern = re.compile(r'^제(\d+)절\s+(.+)', re.MULTILINE)
    article_pattern = re.compile(r'^제(\d+)조(?:의(\d+))?\s*\(([^)]+)\)', re.MULTILINE)
    
    # Find all chapters, sections, and articles in the text
    chapters = list(chapter_pattern.finditer(text))
    sections = list(section_pattern.finditer(text))
    articles = list(article_pattern.finditer(text))
    
    chapter_idx = 0
    section_idx = 0
    
    result = []  # List to store parsed data
    
    current_chapter_num = None
    current_chapter_title = ""
    current_section_title = ""
    
    # Capture article positions for determining content boundaries
    article_starts = [m.start() for m in articles]
    article_starts.append(len(text))
    
    for i, article in enumerate(articles):
        article_num = article.group(1)
        sub_num = article.group(2) if article.group(2) else ""
        article_title = article.group(3).strip()
        
        article_start = article.start()
        article_end = article.end()
        next_article_start = article_starts[i+1]
        
        # Extract content between articles
        content = text[article_end:next_article_start].strip()
        
        # Update chapter information if applicable
        while chapter_idx < len(chapters) and chapters[chapter_idx].start() < article_start:
            current_chapter_num = chapters[chapter_idx].group(1)
            current_chapter_title = chapters[chapter_idx].group(2).strip()
            current_section_title = ""  # Reset section title for new chapter
            chapter_idx += 1
        
        # Update section information if applicable
        while section_idx < len(sections) and sections[section_idx].start() < article_start:
            section_number = sections[section_idx].group(1)
            section_text = sections[section_idx].group(2).strip()
            current_section_title = f"제{section_number}절 {section_text}"  # Combine section number and title
            section_idx += 1
        
        # Construct chapter ID
        if sub_num:
            chapter_id = f"{current_chapter_num}-{article_num}-{sub_num}"
        else:
            chapter_id = f"{current_chapter_num}-{article_num}"
        
        # Construct chapter title
        chapter_title = f"{current_chapter_title} - {current_section_title}" if current_section_title else current_chapter_title
        
        # Build JSON object for the article
        json_obj = {
            "chapter_id": chapter_id,
            "chapter_title": chapter_title,
            "title": article_title,
            "content": content
        }
        
        result.append(json_obj)
    
    return result

# Adjust the input text to ensure proper parsing of chapters and sections
response_content = re.sub(r'(제\d+장[^\n]+?)\s+(제\d+절)', r'\1\n\2', response_content)  # Add newline between chapters and sections

# Parse the immigration law content
parsed_law = parse_immigration_law(response_content)

# Save the parsed data as a JSON file
output_file = r"immigration.json"

with open(output_file, "w", encoding="utf-8") as f:
    json_output = json.dump(parsed_law, f, ensure_ascii=False, indent=2)  # Save JSON with proper formatting

print(f"Parsed data saved to {output_file}")
